In [113]:
#imports
import pyodbc
import pandas as pd

In [114]:
#connect to db
driver = "{ODBC Driver 17 for SQL Server}"
server = "172.20.10.149\PRODUCTION"
database = "stb_production"
username = "Neo.Tech"
password = "Password357"

conn = pyodbc.connect("DRIVER=" + driver
+ ";SERVER=" + server
+ ";DATABASE=" + database
+ ";UID=" + username
+ ";PWD=" + password )
cursor = conn.cursor()

In [115]:
filename = 'BGA Rework - 3136 Units.xlsx'

pallet_col = 'Pallet 1'


excel_data = pd.read_excel('BGA Rework - 3136 Units.xlsx')
# Read the values of the file in the dataframe
df = pd.DataFrame(excel_data, columns=[pallet_col])
# Print the content
# print("The content of the file is:\n", df)

In [129]:
stb_find_str = '\''
stb_find_list = []

stb_found_dict =  {
#    KEY: stb_num_short, VALUE: {
#       stb_num: XXXX,
#       cdsn_iuc: XXXX,
#       pcb_num: XXXX
# }
}

product = '4138'

pallet_stbs = df.iterrows()

# print(df.to_numpy().flatten())
pallet_stbs_list = df.to_numpy().flatten()

print(f'{pallet_col} has {len(pallet_stbs_list)} stbs')

## TEST PARAMS
TEST_FLAG = False
TEST_COUNT = 0
TEST_ITERATION = 7 #TODO: Set this to Zero before running in production

for stb_num_short in pallet_stbs_list:
    if (TEST_FLAG and TEST_COUNT < TEST_ITERATION):
        print(f'--TEST_COUNT-- {TEST_COUNT}')
        TEST_COUNT +=1
    elif (TEST_FLAG):
        break
    # print(f'{stb_num_short}')
    # time.sleep(1)
    for row in cursor.execute(f'''SELECT stb_num FROM stb_production.dbo.production_event
    WHERE stb_num LIKE \'{stb_num_short}%\''''):
        # print(f'>> {row.stb_num}')
        stb_find_str = stb_find_str + row.stb_num + '\',\''
        stb_find_list.append(row.stb_num)
        # temp['found'] = row.stb_num.strip()
        # print(f'>> {stb_find_str}')
    stb_find_list.append(stb_num_short)
    stb_found_dict[stb_num_short.strip()] = {
        "stb_num": 'NOT FOUND',
        "cdsn_iuc": 'NOT FOUND',
        "pcb_num": 'NOT FOUND'
    }




Pallet 1 has 784 stbs


KeyboardInterrupt: 

In [130]:
# Removes last character from string
stb_find_str = stb_find_str[:-2]

print(f' SQL Input String for Like Statement {stb_find_str}')
print(f' SQL Input List Like Statement {stb_find_list}')
# print(f' SQL Input Dict Like Statement {stb_found_dict}')

# for row in cursor.execute(f'''SELECT stb_num FROM stb_production.dbo.production_event
# WHERE stb_num LIKE \'{input_csv_stb_num}%\''''):
#     print(row.stb_num)

# TODO: improve, merge all above stb_nums with ''
print(f'''\n{"KEY_WORD"}\t\t{"STB_NUM"}\t\t\t{"IUC_NUM"}\t\t\t{"PCB_NUM"}''')
for innerrow in cursor.execute(f'''SELECT stb_num, cdsn_iuc, pcb_num FROM stb_production.dbo.production_event
                                    INNER JOIN
                                    device_state_dsd_{product} ON production_event.id_production_event = device_state_dsd_{product}.id_production_event
                                    WHERE        stb_num IN ({stb_find_str})'''):
    # print(f'''{innerrow.stb_num[:-1]}\t\t{innerrow.stb_num}\t\t{innerrow.cdsn_iuc}\t\t{innerrow.pcb_num}''')
    stb_found_dict[innerrow.stb_num.strip()[:-1]] = {
        "stb_num": innerrow.stb_num,
        "cdsn_iuc": innerrow.cdsn_iuc,
        "pcb_num": innerrow.pcb_num
    }
    
# print(f'''keys: {stb_found_dict.keys()}''')
for stb_num_short in stb_found_dict.keys():
    try:
        print(f'''{stb_num_short}\t\t{stb_found_dict[stb_num_short]['stb_num']}\t\t{stb_found_dict[stb_num_short]['cdsn_iuc']}\t\t{stb_found_dict[stb_num_short]['pcb_num']}''')
    except KeyError:
        print(f'''{stb_num_short}\t\t{'NOT FOUND'}\t\t{'NOT FOUND'}\t\t{'NOT FOUND'}''')



 SQL Input String for Like Statement 'S0400067325','S0414004652','S0409773923','S0416891974','S0403420557','S0409828957','S0410762609','S0414301878','S0417320440','S0405204207','S0410651823','S0417537562','S0418753170','S0419570990','S0406495370','S0402347678','S0401835385','S0413103150'
 SQL Input List Like Statement ['S041153204', 'S040789152', 'S0400067325', 'S040006732', 'S040846160', 'S0414004652', 'S041400465', 'S041170035', 'S040864251', 'S0409773923', 'S040977392', 'S0416891974', 'S041689197', 'S0403420557', 'S040342055', 'S0409828957', 'S040982895', 'S0410762609', 'S041076260', 'S0414301878', 'S041430187', 'S0417320440', 'S041732044', 'S0405204207', 'S040520420', 'S0410651823', 'S041065182', 'S0417537562', 'S041753756', 'S0418753170', 'S041875317', 'S0419570990', 'S041957099', 'S040749638', 'S0406495370', 'S040649537', 'S041164170', 'S040869381', 'S040844445', 'S040740848', 'S041175903', 'S0402347678', 'S040234767', 'S0401835385', 'S040183538', 'S041151918', 'S041164234', 'S04